In [1]:
# -*- coding: utf-8 -*-
import numpy as np
from re import sub, findall, MULTILINE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from unicodedata import normalize
from json import loads
from csv import reader,writer,QUOTE_ALL
from sklearn.model_selection import train_test_split
from string import punctuation

from pickle import load
from operator import itemgetter
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier

from sklearn import svm, linear_model
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

#IMPORTING EXTERNAL JUPYTER NOTEBOOKS
import nbimporter
from readability import *
import nltk_linguistic_features as nltk_linguis_features
import utils as utils
from importlib import reload
import sys
reload(sys.modules['readability'])
reload(sys.modules['nltk_linguistic_features'])
reload(sys.modules['utils'])


/usr/local/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


Importing Jupyter notebook from readability.ipynb
Importing Jupyter notebook from nltk_linguistic_features.ipynb
Importing Jupyter notebook from utils.ipynb
Importing Jupyter notebook from readability.ipynb
Importing Jupyter notebook from nltk_linguistic_features.ipynb
Importing Jupyter notebook from utils.ipynb


<module 'utils' from 'utils.ipynb'>

### Readability features
- 111: 'characters', 
- 112: 'complexWords', 
- 113: 'longWords', 
- 114: 'numberSyllables', 
- 115: 'lexicon_count', 
- 116: 'sentence_count', 
- 117: 'flesch_reading_ease', 
- 118: 'smog_index', 
- 119: 'flesch_kincaid_grade', 
- 120: 'coleman_liau_index', 
- 121: 'automated_readability_index', 
- 122: 'difficult_words', 
- 123: 'linsear_write_formula', 
- 124: 'gunning_fog', 
- 125: 'word_count', 
- 126: 'words_per_sent', 
- 127: 'nr_captalized_words', 
- 128: 'nr_per_stopwords', 
- 129: 'count_urls', 

### NLTK features
- 130:'conjunction, coordinating', 
- 131:'numeral, cardinal', 
- 132:'determiner', 
- 133:'foreign word', 
- 134:'preposition or conjunction, subordinating', 
- 135:'adjective or numeral, ordinal', 
- 136:'adjective, comparative', 
- 137:'adjective, superlative', 
- 138:'modal auxiliary', 
- 139:'noun, common, singular or mass', 
- 140:'noun, proper, singular', 
- 141:'noun, proper, plural', 
- 142:'noun, common, plural', 
- 143:'pre-determiner', 
- 144:'genitive marker', 
- 145:'pronoun, personal', 
- 146:'pronoun, possessive', 
- 147:'adverb', 
- 148:'adverb, comparative', 
- 149:'adverb, superlative', 
- 150:'particle', 
- 151:'"to" as preposition/infinitive', 
- 152:'interjection', 
- 153:'verb, base form', 
- 154:'verb, past tense', 
- 155:'verb, present participle or gerund', 
- 156:'verb, past participle', 
- 157:'verb, present tense, not 3rd person singular', 
- 158:'verb, present tense, 3rd person singular', 
- 159:'WH-determiner', 
- 160:'WH-pronoun', 
- 161:'WH-pronoun, possessive', 
- 162:'Wh-adverb'

In [ ]:
#### headline and content csv files contain the following features:
#url = t[0] A
#headline = t[1] B
#content = t[2] C
#publish_date = t[3] D
#author = t[4] E
#freq_basic = t[5] F
#freq_formatting = t[6] G
#freq_forms_inputs = t[7] H
#freq_frames = t[8] I
#freq_images = t[9] J
#freq_audio_video = t[10] K
#freq_links = t[11] L 
#freq_lists = t[12] M
#freq_tables = t[13] N
#freq_styles_semantics = t[14] O
#freq_meta_info = t[15] P
#freq_programming_info = t[16] Q
#freq_ads = t[17] R

#### headlineContent csv file contains the following features (See how change the THIRD column. It will contain the headline + content):
#url = t[0] A
#headline = t[1] B
#headline+content = t[2] C
#publish_date = t[3] D
#author = t[4] E
#freq_basic = t[5] F
#freq_formatting = t[6] G
#freq_forms_inputs = t[7] H
#freq_frames = t[8] I
#freq_images = t[9] J
#freq_audio_video = t[10] K
#freq_links = t[11] L 
#freq_lists = t[12] M
#freq_tables = t[13] N
#freq_styles_semantics = t[14] O
#freq_meta_info = t[15] P
#freq_programming_info = t[16] Q
#freq_ads = t[17] R

### Extract and save features (Readability and NLTK features)

 <b>Function: </b> get_save_Readability_NLTK_features(filename, type_text). This function add (rewrite over the same csv file) the following features: Readability (19 features) and NLKT (33 features). <br />
 <b>Parameters:</b> <br />
 <b>filename</b>: csv file source. There are three type of csv files: headline, content, or a headlineContent csv file (see above to get more details about these type of csv files). This csv file should contains already the following features: urls(1), headline(1), content or headline+content(1), date(1), Webmarkup (14 features), and LIWC (93 features). LIWIC features are optional but highly recommended.
 <br />
 <b>type_text:</b> It defines the type of text (headline, content or headline+content) that will be used to extract readability and NLTK features. 1 = Headline, 2 = Content, 2 = HeadlineContent. Content and HeadlineContent have the same value (2) because <b>it will depend of the type of csv file source</b>. If it is HeadlineContent csv file, so, the third column should contains headline plus content ( string concatenation). But if it is a Content csv file, the third column should contains just the content. FYI: The second column will always contain only the headline in the three types of csv files.  

In [4]:
def get_save_Readability_NLTK_features(filename, type_text):
    list_docs, list_labels, list_urls = [], [], []
    count_urls, count_mentions = 0, 0
    subset = []
    index = 1
    with open(filename, "r") as arq_in:
        reader_in = reader(arq_in, delimiter=',', quoting=QUOTE_ALL)
        for t in reader_in:
            
            if(len(t[0]) > 0):#handling missing features
                #text, count_mentions = remove_mention_corpus()
                
                #get features based on NLP
                text, count_urls = nltk_linguis_features.remove_ulr_corpus(t[type_text])
                tokens = nltk_linguis_features.tokenizer(text)

                pos_tag_numbers   = nltk_linguis_features.part_of_speech_tagger(tokens)
                word_count, words_per_sent = nltk_linguis_features.sentence_level_attr(text,tokens)
                nr_captalized_words  = nltk_linguis_features.count_captalized_words(tokens)
                nr_per_stopwords = nltk_linguis_features.count_per_stopwords(tokens)
                nr_punctuation = nltk_linguis_features.count_punctuation(text,r'['+punctuation+r']+')
                nr_quotes = nltk_linguis_features.count_punctuation(text,r'["\']+')
 
                instance = []
    
                #Checking if LIWIC features were added.
                if( not len(t)>20):
                    print('[Warning] LIWIC features were not extracted. Please extract those manually and update the output cvs file. Remenber this is just optional.')
                    instance = instance + list(np.zeros(93)) #adding zeros because there is no LIWIC features
                    
                #Adding READABILITY features
                readability = []
                if(text.strip()==""):
                    readability = list(np.zeros(14))+ [word_count, words_per_sent] +\
                        [nr_captalized_words, nr_per_stopwords]+\
                        [count_urls]
                else:
                    readability = [characters_stat(text), complexWords_stat(text), longWords_stat(text), numberSyllables_stat(text), lexicon_count_stat(text), sentence_count_stat(text), flesch_reading_ease_stat(text), smog_index_stat(text), flesch_kincaid_grade_stat(text), coleman_liau_index_stat(text), automated_readability_index_stat(text), difficult_words_stat(text), linsear_write_formula_stat(text), gunning_fog_stat(text)] + [word_count, words_per_sent] +\
                        [nr_captalized_words, nr_per_stopwords]+\
                        [count_urls]
                instance = instance + readability  
                if(len(readability)!=19):
                    print("ERROR. Missing readability features-------------------")
                    print(len(instance))
                
                #Adding linguistic features based on NLTK 
                if(text.strip()==""):
                    instance = instance + list(np.zeros(33))
                else:
                    instance = instance + pos_tag_numbers 
             
                if(len(pos_tag_numbers)!=33):
                    print("ERROR. Missing NLTK features-------------------")
                    print(len(pos_tag_numbers))
                instance = t[0:len(t)] + instance
                
                #ERROR. Missing some features
                if(len(instance)!= 163):
                    print('line', index, len(instance))
                    print(len(t))
                subset.append(instance)
                index = index+1
    new_namefile_subset = filename #[:-4]+'_includeLIWC'+'.csv'
    utils.save_csv_file(subset, new_namefile_subset)
    return "completed"



### Create CSV files with preprocessed features

In [111]:
# NYU_Crawled_data dataset
path_politicalNYUCrawled = "/Users/.../Fakenews/paper_fn/Experiments/data/NYU_crawled_data_LIWC_Fixed/"

fake_files = ['fakenews_NYUcrawl_headline.csv','fakenews_NYUcrawl_content.csv','fakenews_NYUcrawl_headlineContent.csv']
msm_files = ['realnews_NYUcrawl_headline.csv', 'realnews_NYUcrawl_content.csv', 'realnews_NYUcrawl_headlineContent.csv']

data = [fake_files, msm_files]
dataLIWC = []
for i in range(len(data)):
    temp = []
    for j in range(len(data[i])):
        temp.append('LIWC2015_' + data[i][j])
    dataLIWC.append(temp)

In [112]:
# Based on headline
print('------------------------headline-------------------------------')
dir_file_test_fake =path_politicalNYUCrawled+ dataLIWC[0][0]
dir_file_test_msm =path_politicalNYUCrawled+ dataLIWC[1][0]
extract_save_Readability_NLTK_features(dir_file_test_fake, 1) 
extract_save_Readability_NLTK_features(dir_file_test_msm, 1) 

------------------------headline-------------------------------


'completed'

In [113]:
print('------------------------content-------------------------------')
dir_file_test_fake =path_politicalNYUCrawled+ dataLIWC[0][1]
dir_file_test_msm =path_politicalNYUCrawled+ dataLIWC[1][1]
extract_save_Readability_NLTK_features(dir_file_test_fake, 2) 
extract_save_Readability_NLTK_features(dir_file_test_msm, 2)# 2 = content 
print('-------------------------headline_content------------------------------')
dir_file_test_fake =path_politicalNYUCrawled+ dataLIWC[0][2]
dir_file_test_msm =path_politicalNYUCrawled+ dataLIWC[1][2]
extract_save_Readability_NLTK_features(dir_file_test_fake, 2) 
extract_save_Readability_NLTK_features(dir_file_test_msm, 2)# 2 = content 

------------------------content-------------------------------
-------------------------headline_content------------------------------


'completed'